# BERT Embedding Generation

This notebook contains generating BERT<sub>BASE</sub> embedding on different pooling strategies.

In [ ]:
import os
import urllib

from google.colab import drive, files
from getpass import getpass

from google.colab import drive

In [ ]:
ROOT = '/content/drive'
GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/recommender/w266-final'
PROJECT_PATH = os.path.join(ROOT, GOOGLE_DRIVE_PATH)

drive.mount(ROOT)

%cd {PROJECT_PATH}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/recommender/w266-final


In [ ]:
import os
import sys
import re
import pandas as pd
import numpy as np
import itertools
import pickle
import random
import tensorflow as tf

from commons.store import PickleStore, NpyStore
from tqdm import tqdm
from IPython.core.display import HTML
from importlib import reload

%load_ext autoreload
%autoreload 2

## 1. Load Pre-filtered Dataset

Load the clean pre-processed dataset.

In [ ]:
amazon = False

if amazon:
  input_pkl = '../dataset/25-65_tokens_grouped_Movies_and_TV_v2.pkl'
else:
  input_pkl = '../dataset/25-65_tokens_grouped_yelp.pkl'

pkl_store = PickleStore(input_pkl)

grouped_reviews_df = pkl_store.load(asPandasDF=True, \
                                    columns=['reviewerID', 'asin', 'overall', 'userReviews', 'itemReviews'])
print(len(grouped_reviews_df))
display(HTML(grouped_reviews_df.head(1).to_html()))

Loading ../dataset/25-65_tokens_grouped_yelp.pkl ...
	... 272296 records
Done!!
272296


In [ ]:
#grouped_reviews = grouped_reviews_df[['userReviews', 'itemReviews', 'overall']].to_numpy()
grouped_reviews = grouped_reviews_df.to_numpy()
grouped_reviews[0]

array(['JHXQEayrDHOWGexs0dCviA', 'KXCXaF5qimmtKKqnPc_LQA', 1.0,
       array(['the dark chocolate gelato is so rich and creamy and seductiveyou must lick it once in your life but their prices recently went up so take your credit card',
       'great pastries and chocolate but why does the service have to be soooo slow and disorganized sigh',
       "great ice cream but are the staff handpicked for their incompetence and total lack of training the service is that bad it's almost embarrassing",
       "great baked goods but please don't remind me 1530 minutes before you close that we are closing at 6 as it's not polite it's not etiquette it's not nice to pressure customers to hurry up thank you",
       "it's really hokey and old but not in a good way in an ancient dirty way a good place to view a cross section of west vancouver ie grey hair it needs to be renovatedthe 'old world charm' is simply not there",
       "the gelato is good not quite to the italian standard but hey you can't h

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 13.4MB/s 
     |████████████████████████████████| 3.3MB 60.4MB/s 
     |████████████████████████████████| 870kB 67.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=cf2950bfc975f613db9119af4e4b379cc742fce7d830250bdf55bd1dd6674181
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import tensorflow as tf

from transformers import BertTokenizer
from transformers import TFBertModel, BertConfig

In [ ]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)


Running on single GPU  /device:GPU:0
Number of accelerators:  1


Using BERT huggingface <img src='https://huggingface.co/front/assets/huggingface_logo.svg' width='20px'> library to load the BERT tokenizer and model.

In [ ]:
bert_model_name = 'bert-base-uncased'

MAX_LEN = 128

config = BertConfig() 
config.output_hidden_states = True # set to True to obtain hidden states

with strategy.scope():
  tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
  user_bert = TFBertModel.from_pretrained(bert_model_name, config=config)
  item_bert = TFBertModel.from_pretrained(bert_model_name, config=config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are in

## 2. Pooling Strategies




### 2.1. Last Hidden State

In [ ]:
def last_hidden_state_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens)[0]
    item_embedding = item_model(item_tokens)[0]

    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)

    #_embeddings[i] = ((dict(user_embedding=user_embedding, 
    #                      item_embedding=item_embedding)), label)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = last_hidden_state_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 150000 of 150000 record(s) -- 100.00%
	Done!
CPU times: user 14h 38min 48s, sys: 21min 50s, total: 15h 39s
Wall time: 17h 12min 4s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)

embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)



Saving ../dataset/embedding/grouped_embedding_0-150000_yelp.npy ...
	... 150000 records
Done!!



### 2.2. Sum Last Four Hidden States

In [ ]:
def sum_last_four_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens).hidden_states
    item_embedding = item_model(item_tokens).hidden_states

    # Sum last four hiddden states
    user_embedding = tf.reduce_sum(user_embedding[-4:], axis=0)
    item_embedding = tf.reduce_sum(item_embedding[-4:], axis=0)

    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = sum_last_four_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 150000 of 150000 record(s) -- 100.00%
	Done!
CPU times: user 10h 45s, sys: 20min 29s, total: 10h 21min 14s
Wall time: 13h 44min 12s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_sumlastfour_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_sumlastfour_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)
  
embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)

Saving ../dataset/embedding/grouped_embedding_sumlastfour_0-150000_yelp.npy ...
	... 150000 records
Done!!


### 2.3. Sum Last Twelve Hidden States

In [ ]:
def sum_last_twelve_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens).hidden_states
    item_embedding = item_model(item_tokens).hidden_states

    # Sum last twelve hiddden states
    user_embedding = tf.reduce_sum(user_embedding[-12:], axis=0)
    item_embedding = tf.reduce_sum(item_embedding[-12:], axis=0)

    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = sum_last_twelve_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 150000 of 150000 record(s) -- 100.00%
	Done!
CPU times: user 10h 14min 57s, sys: 21min 37s, total: 10h 36min 35s
Wall time: 13h 54min 38s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_sumlasttwelve_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_sumlasttwelve_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)
  
embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)

Saving ../dataset/embedding/grouped_embedding_sumlasttwelve_0-150000_yelp.npy ...
	... 150000 records
Done!!


### 2.4. Second-To-Last Hidden State

In [ ]:
def second_to_last_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens).hidden_states
    item_embedding = item_model(item_tokens).hidden_states
    
    # Second-to-last hiddden states
    user_embedding = user_embedding[-2]
    item_embedding = item_embedding[-2]
    
    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = second_to_last_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 150000 of 150000 record(s) -- 100.00%
	Done!
CPU times: user 10h 20min 45s, sys: 20min 35s, total: 10h 41min 20s
Wall time: 13h 56min 54s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_secondtolast_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_secondtolast_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)
  
embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)

Saving ../dataset/embedding/grouped_embedding_secondtolast_0-150000_yelp.npy ...
	... 150000 records
Done!!


### 2.5. First Layer Hidden State

In [ ]:
def first_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens).hidden_states
    item_embedding = item_model(item_tokens).hidden_states
    
    # First embedding
    user_embedding = user_embedding[0]
    item_embedding = item_embedding[0]
    
    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = first_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 150000 of 150000 record(s) -- 100.00%
	Done!
CPU times: user 10h 15min 7s, sys: 21min 18s, total: 10h 36min 25s
Wall time: 13h 52min 36s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_first_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_first_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)
  
embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)

### 2.6. Concat Last Four Hidden States

In [ ]:
def concat_last_four_embedding(samples, tokenizer, user_model, item_model, max_len=128):

  def tokenize(reviews):
    return tokenizer(list(reviews), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

  total = len(samples)
  _embeddings = np.empty(len(samples), dtype=object)

  for i, reviews in enumerate(samples):

    user_tokens = tokenize(reviews[3])
    item_tokens = tokenize(reviews[4])
    
    reviewerID = reviews[0]
    asin = reviews[1]
    label = reviews[2]
    
    user_embedding = user_model(user_tokens).hidden_states
    item_embedding = item_model(item_tokens).hidden_states

    # Concat last four hiddden states
    user_embedding = tf.concat(user_embedding[-4:], axis=2)
    item_embedding = tf.concat(item_embedding[-4:], axis=2)

    user_embedding = tf.stack([user_embedding])
    item_embedding = tf.stack([item_embedding])

    user_embedding = tf.keras.layers.GlobalAveragePooling2D()(user_embedding)
    item_embedding = tf.keras.layers.GlobalAveragePooling2D()(item_embedding)
    
    _embeddings[i] = dict(reviewerID=reviewerID, asin=asin, 
                            user_embedding=user_embedding, 
                            item_embedding=item_embedding,
                            label=label)

    
    print(f'\rEmbedding... {i+1} of {total} record(s) -- {(i+1)/total*100:.2f}%', end='')
  
  print('\n\tDone!')
  return _embeddings

In [ ]:
%%time
start = 0
end = 150000
embeddings = concat_last_four_embedding(grouped_reviews[start:end], tokenizer, user_bert, item_bert)

Embedding... 50000 of 50000 record(s) -- 100.00%
	Done!
CPU times: user 4h 48min 47s, sys: 7min 16s, total: 4h 56min 4s
Wall time: 5h 32min 36s


In [ ]:
embedding_dir = '../dataset/embedding/'
if not os.path.exists(embedding_dir):
    os.makedirs(embedding_dir)

if amazon:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_concatlastfour_',str(start),'-',str(end),'_Movies_and_TV.npy'])
else:
  embedding_npy = ''.join([embedding_dir, 'grouped_embedding_concatlastfour_',str(start),'-',str(end),'_yelp.npy'])

if os.path.exists(embedding_npy):
  os.remove(embedding_npy)
  
embedding_store = NpyStore(embedding_npy)
embedding_store.write(embeddings)

Saving ../dataset/25-65_tokens/embedding/grouped_embedding_concatlastfour_0-50000_Movies_and_TV.npy ...
	... 50000 records
Done!!
